In [5]:
#from __future__ import division #Doesn't work even though it's suppose to do division using floats
import sqlite3
import pandas as pd
import numpy as np
conn = sqlite3.connect("studies101And102.db")
curs = conn.cursor()

curs.execute("CREATE TABLE IF NOT EXISTS q1StronglyDisagreeOrDisagree AS SELECT count(*) from responses_table WHERE question_id='Q1' AND response<3")

curs.execute("CREATE TABLE IF NOT EXISTS q1GroupBySexStronglyDisagreeOrDisagree AS SELECT Sex, count(*) from responses_table INNER JOIN respondents_table on respondents_table.respondent_id = responses_table.respondent_id AND respondents_table.study_id = responses_table.study_id WHERE question_id='Q1' AND response<3 GROUP BY Sex")

curs.execute("CREATE TABLE IF NOT EXISTS response_cat_table (response INTEGER PRIMARY KEY, response_cat STRING);")
curs.execute("INSERT OR IGNORE INTO response_cat_table (response, response_cat) VALUES (1, 'Disagree or strongly disagree'), (2, 'Disagree or strongly disagree'), (3, 'Neutral'), (4, 'Agree or strongly agree'), (5, 'Agree or strongly agree'), (6, 'Do not know or not sure');")
curs.execute("CREATE TABLE IF NOT EXISTS q1GroupBySexAndResponseCat AS SELECT Sex, response_cat, count(*) from responses_table INNER JOIN respondents_table on respondents_table.respondent_id = responses_table.respondent_id AND respondents_table.study_id = responses_table.study_id INNER JOIN response_cat_table on responses_table.response = response_cat_table.response WHERE question_id='Q1' AND responses_table.response<6 GROUP BY Sex, response_cat")

#curs.execute("CREATE TABLE IF NOT EXISTS q3CountEachOption AS SELECT question_id, count(*) from responses_table WHERE question_id LIKE 'Q3%' AND response=1 GROUP BY question_id")
curs.execute("CREATE TABLE IF NOT EXISTS q3ProportionOfEachOption AS SELECT question_id, count(*)/20.0 proportion from responses_table WHERE question_id LIKE 'Q3%' AND response=1 GROUP BY question_id")

curs.execute("CREATE TABLE IF NOT EXISTS q3ProportionOfEachOptionGroupBySex AS SELECT question_id, Sex, sum(response)/10.0 AS proportion from responses_table INNER JOIN respondents_table on respondents_table.respondent_id = responses_table.respondent_id AND respondents_table.study_id = responses_table.study_id WHERE question_id LIKE 'Q3%' GROUP BY question_id, Sex")

studyId = 102
curs.execute("SELECT respondent_id, study_id, Number_of_children FROM respondents_table WHERE study_id="+str(studyId))
numberOfChildrenCategorised = curs.fetchall()
curs.execute("CREATE TABLE IF NOT EXISTS number_of_children_categorised (respondent_id, study_id, number_of_children_cat STRING);")
for row in numberOfChildrenCategorised:
    if row[2]==0:
        row=(row[0],row[1],'No children')
    elif row[2]<3:
        row=(row[0],row[1],'1 or 2 children')
    elif row[2]>2:
        row=(row[0],row[1],'3 or more children')
    #print(row)
    curs.execute("INSERT OR IGNORE INTO number_of_children_categorised (respondent_id, study_id, number_of_children_cat) VALUES (?,?,?)", row)
curs.execute("CREATE TABLE IF NOT EXISTS numberOfRespondentsChildrenCat AS SELECT number_of_children_cat, count(*) from number_of_children_categorised GROUP BY number_of_children_cat")

conn.commit()
conn.close()